In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [8]:
import asyncio
import nest_asyncio
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from IPython.display import Markdown, display

nest_asyncio.apply()

class ManualMCPBridge:
    def __init__(self, server_script, cwd):
        self.server_script = server_script
        self.cwd = cwd
        self.process = None
        self.request_id = 1
        # ADDED: Lock to prevent concurrent reading from the same pipe
        self.lock = asyncio.Lock()

    async def start(self):
        self.process = await asyncio.create_subprocess_exec(
            "node", self.server_script,
            stdin=asyncio.subprocess.PIPE,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
            cwd=self.cwd
        )
        async def log_stderr():
            while True:
                line = await self.process.stderr.readline()
                if not line: break
                # Server logs go to console for transparency
                print(f"🛠️ [Server]: {line.decode().strip()}", flush=True)
        asyncio.create_task(log_stderr())

        # Initial handshake
        await self.send_request("initialize", {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {"name": "manual-bridge", "version": "1.0.0"}
        })

    async def send_request(self, method, params):
        # We wrap everything in a lock so tool calls don't collide
        async with self.lock:
            cur_id = self.request_id
            self.request_id += 1
            payload = {"jsonrpc": "2.0", "id": cur_id, "method": method, "params": params}

            self.process.stdin.write(json.dumps(payload).encode() + b"\n")
            await self.process.stdin.drain()

            while True:
                line = await self.process.stdout.readline()
                if not line: return None
                try:
                    resp = json.loads(line.decode())
                    # Only return if the ID matches our current request
                    if resp.get("id") == cur_id:
                        return resp.get("result")
                except:
                    # Ignore non-JSON lines (logs accidentally sent to stdout)
                    continue

    async def list_tools(self):
        res = await self.send_request("tools/list", {})
        return res.get("tools", [])

    async def call_tool(self, name, args):
        res = await self.send_request("tools/call", {"name": name, "arguments": args})
        content = res.get("content", [])
        return "".join([c.get("text", "") for c in content if c.get("type") == "text"])

    def stop(self):
        if self.process:
            self.process.terminate()

async def run_manual_agent(user_idea):
    repo_path = "/content/skinguide-mcp-server"
    server_path = os.path.join(repo_path, "dist/index.js")

    bridge = ManualMCPBridge(server_path, repo_path)
    await bridge.start()

    try:
        mcp_tools = await bridge.list_tools()
        print(f"✅ Tools Ready: {[t['name'] for t in mcp_tools]}")

        langchain_tools = []
        for t_def in mcp_tools:
            t_name = t_def['name']
            t_desc = t_def['description']

            def create_tool(name, desc):
                @tool
                async def dynamic_tool(**kwargs):
                    """Dynamic MCP Tool Wrapper"""
                    print(f"  🔍 Calling: {name} with {kwargs}")
                    return await bridge.call_tool(name, kwargs)

                dynamic_tool.name = name
                dynamic_tool.description = desc
                return dynamic_tool

            langchain_tools.append(create_tool(t_name, t_desc))

        llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
        agent = create_react_agent(llm, langchain_tools)

        print(f"🤖 AI is researching and writing: '{user_idea}'...")
        prompt = (
            f"You are a professional Beauty & Skincare Blogger.\n"
            f"User Goal: '{user_idea}'.\n"
            f"1. Use the tools to find real data.\n"
            f"2. Write a comprehensive Markdown blog post."
        )

        inputs = {"messages": [HumanMessage(content=prompt)]}
        final_text = ""
        async for chunk in agent.astream(inputs, stream_mode="values"):
            msg = chunk["messages"][-1]
            if msg.type == "ai" and not msg.tool_calls:
                final_text = msg.content

        return final_text

    finally:
        bridge.stop()




🛠️ [Server]: [skinguide-mcp] 🚀 Initializing SkinGuide MCP Server
🛠️ [Server]: [skinguide-mcp] ✅ Transport initialized (stdio)
🛠️ [Server]: [skinguide-mcp] 🎉 Server connected and listening on stdio
✅ Tools Ready: ['search_products', 'get_skin_type_info', 'list_skin_types', 'get_product_types']
🤖 AI is researching and writing: 'A daily routine for oily acne-prone skin with product recommendations'...


/tmp/ipython-input-219/2542587143.py:110: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, langchain_tools)


  🔍 Calling: list_skin_types with {}
  🔍 Calling: get_product_types with {}
🛠️ [Server]: [skinguide-mcp] 📋 Executing list_skin_types
🛠️ [Server]: [skinguide-mcp] 🏷️  Executing get_product_types
  🔍 Calling: search_products with {}
🛠️ [Server]: [skinguide-mcp] 📦 Executing search_products with params: {"country":"US","limit":50}

📝 GENERATED BLOG POST



# A Comprehensive Daily Skincare Routine for Oily, Acne-Prone Skin

Oily, acne-prone skin can be quite challenging to manage. The excess oil production often leads to clogged pores and breakouts. However, with the right skincare routine, you can maintain a clear and healthy complexion. In this blog post, I'll walk you through an effective daily skincare routine specifically designed for oily and acne-prone skin types, along with some product recommendations to help you achieve your skincare goals.

## Understanding Your Skin Type

The Baumann Skin Type system categorizes skin into 16 types based on four parameters: Dry vs. Oily, Sensitive vs. Resistant, Pigmented vs. Non-pigmented, and Wrinkled vs. Tight. For oily and acne-prone skin, the most relevant types are OSPT (Oily, Sensitive, Pigmented, Tight) and ORPT (Oily, Resistant, Pigmented, Tight).

### Characteristics of OSPT Skin Type
- **Oily and Sensitive:** Prone to acne and irritation.
- **Pigmented:** May develop post-inflammatory hyperpigmentation (dark spots) after acne heals.
- **Tight:** Firm skin with minimal aging concerns.

### Characteristics of ORPT Skin Type
- **Oily and Resilient:** Can handle stronger skincare actives.
- **Pigmented:** Similar pigmentation concerns as OSPT.
- **Tight:** Firm and youthful appearance.

## Daily Skincare Routine

### Morning Routine

1. **Cleanser**
   - Use a gentle cleanser to remove excess oil and impurities without stripping the skin.
   - **Recommendation:** [Aveeno Clear Complexion Foaming Oil-Free Facial Cleanser](https://www.amazon.com/dp/B005IHF3O8/?tag=robertoz-20) - Contains soy extract and salicylic acid to treat and prevent breakouts.

2. **Toner**
   - A toner can help balance the skin's pH and control oil production.
   - **Recommendation:** [Neutrogena Alcohol-Free Gentle Daily Facial Toner](https://www.amazon.com/dp/B0BNC5ZF3T/?tag=robertoz-20) - Refreshes the skin while gently removing impurities.

3. **Serum**
   - Opt for a lightweight serum with antioxidants to protect your skin.
   - **Recommendation:** [The Ordinary 100% Organic Cold-Pressed Moroccan Argan Oil](https://amzn.to/4r14gzm) - Hydrates and nourishes without clogging pores.

4. **Moisturizer**
   - Choose an oil-free, non-comedogenic moisturizer to hydrate the skin.
   - **Recommendation:** [Good Molecules Squalane Oil](https://www.amazon.com/dp/B09Q2XYMLM/?tag=robertoz-20) - Provides hydration and calms the skin.

5. **Sunscreen**
   - Essential for protecting the skin from UV damage and preventing dark spots.
   - **Recommendation:** [Mineral Sunscreen Setting Powder SPF 50](https://amzn.to/3Ly8sYg) - Offers sun protection with a matte finish.

### Evening Routine

1. **Cleanser**
   - Use the same cleanser as in your morning routine to remove makeup and impurities.

2. **Exfoliation** (2-3 times a week)
   - Exfoliate to remove dead skin cells and unclog pores.
   - **Recommendation:** [Neutrogena Oil-Free Pink Grapefruit Pore Cleansing Acne Wash](https://amzn.to/4jTPHLt) - Contains salicylic acid for gentle exfoliation.

3. **Toner**
   - Apply your toner as in the morning routine.

4. **Treatment**
   - Use a targeted acne treatment to address breakouts.
   - **Recommendation:** [Clean & Clear Persa-Gel 10 Oil-Free Acne Spot Treatment](https://amzn.to/4aTFj3m) - Contains benzoyl peroxide to reduce acne lesions.

5. **Moisturizer**
   - Apply the same moisturizer as used in the morning.

### Additional Tips

- **Oil-Control Products:** Incorporate blotting sheets or oil-control powders to manage shine throughout the day.
- **Hydration:** Drink plenty of water to keep your skin hydrated from within.
- **Diet:** Maintain a balanced diet rich in antioxidants and low in sugars to support overall skin health.

By following this routine consistently, you can manage oily skin and reduce the frequency and severity of acne breakouts. Tailor the products and steps to suit your skin's specific needs, and consider consulting with a dermatologist for personalized advice.

In [ ]:
# --- EXECUTE ---
user_topic = "A daily routine for oily acne-prone skin with product recommendations"

try:
    blog_post = asyncio.run(run_manual_agent(user_topic))

    print("\n" + "="*40)
    print("📝 GENERATED BLOG POST")
    print("="*40 + "\n")
    display(Markdown(blog_post))

except Exception as e:
    print(f"❌ Failed: {e}")